<h3>auctions collection</h3>

In [5]:
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np

In [2]:
auction_list = []

In [3]:
start = time.time()

for i in range(1,312):
    url = 'https://gratka.pl/nieruchomosci/dzialki-grunty/budowlana?page=' + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = soup.find_all(class_="teaserEstate__anchor")
    for link in links:
        auction_list.append(link['href'])
#     time.sleep(1)
#     if i%50 == 0:
#         time.sleep(5)
        
stop = time.time()
print((stop-start)//60,'min.')

11.0 min.


In [4]:
len(auction_list)

9937

<h3>saving auction list</h3>

In [5]:
with open('real_estate_links.txt','w') as file:
    for link in auction_list:
        file.write(link)
        file.write('\n')

<h3>loading auction list</h3>

In [6]:
auction_list = []

In [7]:
with open('real_estate_links.txt','r') as file:
    for line in file:
        auction_list.append(line[:-1])

<h3>collecting auction details</h3>

In [8]:
data = []

In [9]:
start = time.time()

i = 0
for url in auction_list:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
    except:
        continue
    # location
    try:
        location = soup.find_all('a',class_="parameters__locationLink")
        city = location[0].text
    except:
        city = ''
    try:
        district = location[1].text
    except:
        district = ''
    try:
        voivodeship = location[2].text
    except:
        voivodeship = ''
    # coordinates
    try:
        scripts = soup.find(id='leftColumn').find_all('script')
        localization_y = re.findall(r'(\"\w+szerokosc.*?\"):(\d*[.\d]*)', scripts[6].contents[0])[0][1]
    except:
        localization_y = ''
    try:
        localization_x = re.findall(r'(\"\w+dlugosc.*?\"):(\d*[.\d]*)', scripts[6].contents[0])[0][1]
    except:
        localization_x = ''
    # parameters
    try:
        parameters = soup.find_all('ul',class_="parameters__rolled")
        area = re.findall(r'(Powierzchnia).*\n(.*)m', parameters[0].text)
        area = area[0][1].replace(' ','')
        area = area.replace(',','.')
    except:
        area = ''
    try:
        sewers = re.findall(r'(Kanalizacja).*\n(.*)', parameters[0].text)[0][1]
    except:
        sewers = ''
    try:
        shape = re.findall(r'(Kształt działki).*\n(.*)', parameters[0].text)[0][1]
    except:
        shape = ''
    try:
        road = re.findall(r'(Droga dojazdowa).*\n(.*)', parameters[0].text)[0][1]
    except:
        road = ''
    try:
        fence = re.findall(r'(Ogrodzenie działki).*\n(.*)', parameters[0].text)[0][1]
    except:
        fence = ''
    try:
        water = re.findall(r'(Woda).*\n(.*)', parameters[0].text)[0][1]
    except:
        water = ''
    try:
        electricity = re.findall(r'(Prąd).*\n(.*)', parameters[0].text)[0][1]
    except:
        electricity = ''
    try:
        gas = re.findall(r'(Gaz).*\n(.*)', parameters[0].text)[0][1]
    except:
        gas = ''
    # price
    try:
        price_info = soup.find_all('span',class_="priceInfo__value")
        price = price_info[0].text.replace(' ','').replace('\n','').replace(',','.')[:-2]
    except:
        price = ''
    
    # data collection
    data.append([city,
                 district,
                 voivodeship,
                 localization_y,
                 localization_x,
                 area,
                 sewers,
                 shape,
                 road,
                 fence,
                 water,
                 electricity,
                 gas,
                 price])
    
    i += 1
    if i%500 == 0:
        print(int(i/len(auction_list)*100),'%')
    
#     time.sleep(0.5)
        
stop = time.time()
print((stop-start)//60,'min.')

5 %
10 %
15 %
20 %
25 %
30 %
35 %
40 %
45 %
50 %
55 %
60 %
65 %
70 %
75 %
80 %
85 %
90 %
95 %
151.0 min.


<h3>saving auction details</h3>

In [10]:
with open('real_estate_details_3.csv','w') as file:
    file.write('city,district,voivodeship,localization_y,localization_x,area,sewers,shape,road,fence,water,electricity,gas,price\n')
    for line in data:
        file.write(','.join(line) + '\n')